In [1]:
%load_ext autoreload
%autoreload 1

import sys
sys.path.append("../../utils/")

from GraphAPI import GraphCreator
from graph_helpers import *

%aimport GraphAPI
%aimport graph_helpers

## Generating Graph from Entry Point

1. We initialize our GraphCreator class and check how many new nodes we will need to query. 

In [9]:
gc = GraphCreator("Sonata form")
print("Number of Links to Search:", len(gc.next_links))
print(gc.intro_nodes)

Number of Links to Search: 1023
{'Musical form': True, 'Classical music era': True, 'Movement (music)': True, 'Tonality': True, 'Exposition (music)': True, 'Musical development': True, 'Recapitulation (music)': True, 'Introduction (music)': True, 'Coda (music)': True, 'Sonata': True, 'Symphony': True, 'Concerto': True, 'String quartet': True, 'Musical analysis': True}


2. We query all the nodes linked to/from the entry point (expand our network one level for each node).

In [10]:
gc.expand_network(group_size=2, timeout=5, log_progress=False)

3. Since some nodes will likely have linked to articles through a redirect link, we need to traverse our graph and ensure that all redirects are assigned to the correct nodes. Once all redirects have been dealt with, we remove any old redirect node. 

In [11]:
gc.redraw_redirects()

4. Edges are weighted by how many categories two connected nodes have in common. Once we have all our nodes, and we have dealt with redirects, we can add edge weights for our entire graph. 

In [12]:
gc.update_edge_weights()
gc.get_edge_weights().head()

,source_node,target_node,edge_weight
0,Wolfgang Amadeus Mozart,Ludwig van Beethoven,22
1,Camille Saint-Saëns,Charles-Valentin Alkan,22
2,Ludwig van Beethoven,Wolfgang Amadeus Mozart,22
3,Charles-Valentin Alkan,Camille Saint-Saëns,22
4,Camille Saint-Saëns,Claude Debussy,21


# Getting Our Feature Set

There are two options when generating the feature set:

1. we can generate a standard feature set with only the features themselves. To do this, have the `rank` parameter set to `False`.
2. We can generate a ranked feature set (set `rank` equal to `True`). For each parameter, this will rank them in order of _best_ to _worst_ (this could be ascending or descending, depending on the context of the feature).

After running `get_features_df`, the feature set will be saved in the GraphCreator instance as `feature_df`

In [13]:
features_df = gc.get_features_df(rank=False)

In [14]:
features_df.head()

,node,degree,category_matches_with_source,in_edges,out_edges,centrality,page_rank,adjusted_reciprocity,shortest_path_length_from_entry,shortest_path_length_to_entry,primary_link
0,Classical music,10901,0,9987.0,914.0,0.021728,0.025006,626.0,2.0,1.0,0
1,Baroque,8202,1,7672.0,530.0,0.005908,0.025819,406.0,2.0,2.0,0
2,Johann Sebastian Bach,5497,1,4626.0,871.0,0.139178,0.010322,992.0,2.0,2.0,0
3,19th century,5369,0,3718.0,1651.0,0.000536,0.009006,448.0,2.0,1.0,0
4,Orchestra,4934,0,4588.0,346.0,0.017863,0.009867,272.0,1.0,2.0,0


## Similarity Rank

Two articles are more similar the more categories they share and the closer they are to each other. 

In [16]:
gc.rank_similarity()
gc.features_df[[
    "node", 
    "category_matches_with_source", 
    "primary_link", 
    "shortest_path_length_from_entry",
    "shortest_path_length_to_entry",
    "similarity_rank", 
]].sort_values(["similarity_rank", "category_matches_with_source", "shortest_path_length_from_entry"], ascending=False).head(50)
# gc.features_df[gc.features_df.category_matches_with_source == 1]

,node,category_matches_with_source,primary_link,shortest_path_length_from_entry,shortest_path_length_to_entry,similarity_rank
63,Sonata form,1,0,0.0,0.0,inf
502,Sonata cycle,2,0,2.0,2.0,1.000000
77,Musical form,1,1,2.0,2.0,1.000000
98,Tonality,1,1,2.0,2.0,1.000000
244,Musical development,1,1,2.0,2.0,1.000000
23,Symphony,0,1,1.0,1.0,1.000000
24,String quartet,0,1,1.0,1.0,1.000000
34,Concerto,0,1,1.0,1.0,1.000000
44,Movement (music),0,1,1.0,1.0,1.000000
70,Sonata,0,1,1.0,1.0,1.000000


In [ ]:
gc.intro_nodes

# Basic Plotting

In [ ]:
sns.pairplot(features_df)

In [ ]:
sns.heatmap(features_df.corr())

# Intro Node Values

In [ ]:
intro_nodes = gc.intro_nodes
intro_nodes_df = None
for node in intro_nodes:
    if df is None:
        intro_nodes_df = pd.DataFrame(features_df[features_df.node == node])
    else:
        intro_nodes_df = pd.concat([intro_nodes_df, pd.DataFrame(features_df[features_df.node == node])])
    
intro_nodes_df

In [ ]:
sns.pairplot(intro_nodes_df, hue="node")